In [1]:
import pyrealsense2 as rs
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
import time
import cv2 as cv

In [2]:
RES_WIDTH = 640
RES_HEIGHT = 480

# Gets the pipeline from the realsense
pipe = rs.pipeline()

# Configuration for the camera
config = rs.config()
# Gets a 30 fps depth stream using a z16 format
config.enable_stream(rs.stream.depth, RES_WIDTH, RES_HEIGHT, rs.format.z16, 30)

# Starts the pipeline
pipe.start(config)

In [3]:
# Ascii gradient from white to black
ascii_grad = ' .:-=+*#%@ '

REDUCTION_SCALE = 10
ASCII_WIDTH = RES_WIDTH // REDUCTION_SCALE
ASCII_HEIGHT = RES_HEIGHT // REDUCTION_SCALE

# Loops forever
while True:
    # # Clears output
    clear_output(wait=True)
    
    # Waits and obtains the next camera frame
    frames = pipe.wait_for_frames()
    # Gets the depth data from the frames
    depth = frames.get_depth_frame()
    
    # Gets data as np array
    depth_image = np.asanyarray(depth.get_data())
    
    # Resizes the image
    depth_image = cv.resize(depth_image, (ASCII_WIDTH, ASCII_HEIGHT))
    
    # Converts 16 bit data to 8 bit data
    depth_8bit = cv.convertScaleAbs(depth_image, alpha=0.05)
    
    # Creates a remapping function and vectorizes it
    def get_char(val):
        # Scales the value to the ascii gradient
        return ascii_grad[int(val / 255 * (len(ascii_grad) - 1))]
    apply_func = np.vectorize(get_char)
    
    # Applys the function to the image
    res = apply_func(depth_8bit)
    
    print(depth_8bit)
    
    # Prints the image
    print('\n'.join([''.join(row) for row in res]))
    

[[  0   0 175 ... 219 216 217]
 [  0   0 168 ... 226 231 238]
 [  0   0 173 ... 241 244 245]
 ...
 [  0   0  49 ...   0   0   0]
 [  0   0  48 ...   0   0   0]
 [  0   0   0 ...   0   0   0]]
  ******** =+-*+     ..... ##******************=.#%%%%%%%%%%%%%%
  ******#*-==***    .......#####************#***+-##%%%%%%%%%%@@
  ****#### ==*#      ......-######%###*********+++####%%@@@@@@@@
  ****##** == #-    .......#######%%%%%%%##*#**++++###%%%%@@@@@@
  *****#** ==+#   .........%######%%%%%@%%%%%%**++-##%%%%%%%@   
  ########===###    ....... %%%%%%%%%%%@%%%%%%% -*%%%%%%%%%@@   
 -######## == =     ...... @@@@%%%%%%%%@@@%%@@@@@@  @@@@@@@@@  @
 -########+== %%%   ......%%%%@%%%%%%@@@@ ##@@       @@ @@@@@@@%
 -######## ==        .....%%%%@%%%%%%%%@@%##*@@@#       @@@@@@@ 
 -########.==       .....     %%%%%%%%%@@@#*@+   @         *=%%@
 -########    ...........     %%%%%%%=  .::.#            @    #@
  #######   ..............    %#%%%%   :::::.               -   
  ######  : ..............::